<a href="https://colab.research.google.com/github/Zia-Ul-Hasan/NLP_SEMACNTIC_PRODUCT_SEARCH/blob/main/bert_cosine_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas numpy sentence-transformers --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import re
from sentence_transformers import SentenceTransformer


In [4]:
import kagglehub

path = kagglehub.dataset_download("deeguy/wayfair-wands-dataset")

print("Path to dataset files:", path)

100%|██████████| 18.6M/18.6M [00:00<00:00, 98.4MB/s]


Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/deeguy/wayfair-wands-dataset/versions/1


In [5]:
products_df = pd.read_csv('/content/product.csv', sep='\t')
queries_df = pd.read_csv('/content/query.csv',sep='\t')
labels_df = pd.read_csv('/content/label.csv',sep='\t')


merged_df = labels_df.merge(queries_df, on='query_id', how='left')

merged_df = merged_df.merge(products_df, on='product_id', how='left')

merged_df[['query', 'product_name', 'label']].head()


,query,product_name,label
0,salon chair,21.7 '' w waiting room chair with wood frame,Exact
1,salon chair,22.5 '' wide polyester side chair,Irrelevant
2,salon chair,24.4 '' w metal lounge chair with metal frame,Exact
3,salon chair,25 '' wide faux leather manual swivel standard...,Exact
4,salon chair,27.6 '' w antimicrobial leather seat waiting r...,Exact


In [6]:
print(products_df.columns)
print(queries_df.columns)
print(labels_df.columns)


Index(['product_id', 'product_name', 'product_class', 'category hierarchy',
       'product_description', 'product_features', 'rating_count',
       'average_rating', 'review_count'],
      dtype='object')
Index(['query_id', 'query', 'query_class'], dtype='object')
Index(['id', 'query_id', 'product_id', 'label'], dtype='object')


In [7]:
import re

def clean_text(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', ' ', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Apply the cleaning function
merged_df['clean_query'] = merged_df['query'].apply(clean_text)
merged_df['clean_product'] = merged_df['product_name'].apply(clean_text)

# Preview cleaned data
merged_df[['query', 'clean_query', 'product_name', 'clean_product']].head()


,query,clean_query,product_name,clean_product
0,salon chair,salon chair,21.7 '' w waiting room chair with wood frame,21 7 w waiting room chair with wood frame
1,salon chair,salon chair,22.5 '' wide polyester side chair,22 5 wide polyester side chair
2,salon chair,salon chair,24.4 '' w metal lounge chair with metal frame,24 4 w metal lounge chair with metal frame
3,salon chair,salon chair,25 '' wide faux leather manual swivel standard...,25 wide faux leather manual swivel standard re...
4,salon chair,salon chair,27.6 '' w antimicrobial leather seat waiting r...,27 6 w antimicrobial leather seat waiting room...


In [8]:
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import torch

# Load model
model = SentenceTransformer('all-mpnet-base-v2')

batch_size = 256

def batch_encode(texts):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        batch_embeddings = model.encode(batch, convert_to_tensor=True)
        embeddings.extend(batch_embeddings)
    return embeddings

query_embeddings = batch_encode(merged_df['clean_query'].tolist())
product_embeddings = batch_encode(merged_df['clean_product'].tolist())

semantic_scores = [util.cos_sim(q, p).item() for q, p in zip(query_embeddings, product_embeddings)]
merged_df['semantic_score'] = semantic_scores


100%|██████████| 912/912 [03:37<00:00,  4.19it/s]


In [10]:
from sentence_transformers.util import cos_sim

# Calculate cosine similarity for each pair
semantic_scores = [cos_sim(q, p).item() for q, p in zip(query_embeddings, product_embeddings)]

# Add similarity scores to the DataFrame
merged_df['semantic_score'] = semantic_scores

merged_df[['query', 'product_name', 'label', 'semantic_score']].head()


,query,product_name,label,semantic_score
0,salon chair,21.7 '' w waiting room chair with wood frame,Exact,0.555002
1,salon chair,22.5 '' wide polyester side chair,Irrelevant,0.398499
2,salon chair,24.4 '' w metal lounge chair with metal frame,Exact,0.532462
3,salon chair,25 '' wide faux leather manual swivel standard...,Exact,0.468107
4,salon chair,27.6 '' w antimicrobial leather seat waiting r...,Exact,0.562726


In [11]:
from sentence_transformers import CrossEncoder

cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
top_pairs = list(zip(merged_df['clean_query'], merged_df['clean_product']))
merged_df['re_rank_score'] = cross_encoder.predict(top_pairs)


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.33k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

In [17]:
top_k = 5
top_results = []

for query_id, group in merged_df.groupby('query_id'):
    top_group = group.sort_values(by='re_rank_score', ascending=False).head(top_k)
    top_results.append(top_group)

top_k_df = pd.concat(top_results)
print(top_k_df)

            id  query_id  product_id    label        query     query_class  \
109        109         0       33690  Partial  salon chair  Massage Chairs   
108        108         0       33689  Partial  salon chair  Massage Chairs   
53          53         0        2187  Partial  salon chair  Massage Chairs   
104        104         0        7468    Exact  salon chair  Massage Chairs   
80          80         0        7465    Exact  salon chair  Massage Chairs   
...        ...       ...         ...      ...          ...             ...   
201127  201694       487       40818  Partial   rack glass  Candle Holders   
201599  202166       487       37499  Partial   rack glass  Candle Holders   
201086  201653       487       16930  Partial   rack glass  Candle Holders   
201601  202168       487       28236  Partial   rack glass  Candle Holders   
201281  201848       487       34600  Partial   rack glass  Candle Holders   

                                             product_name  \
10